# File Contents Converter

### バイナリファイル内の文字列を任意に置換するプログラム

#### 01. read確認

In [2]:
fr = open('test_01.DMP', 'rb')
read_data = fr.read()
print(read_data)
fr.close()

b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?'


#### 02. write確認

In [3]:
fw = open('write_01.DMP', 'wb')
fw.write(read_data)
fw.close()

#### 03. 置換の確認

In [5]:
for i in (b'\x00', b'\x10', b'\x20', b'\x30'):
    read_data = read_data.replace(i, b'\xAA')
fc = open('convert_01.DMP', 'wb')
fc.write(read_data)
fc.close()

#### 04. バイナリファイルを指定byteずつreadする

In [3]:
# 指定したファイルのバイトサイズを調べる
import os.path
file_size = os.path.getsize('./test_01.DMP')

64

In [3]:
# ファイルサイズを調べなくてもEOF検出で行ける
with open('./test_01.DMP', 'rb') as fr:
    while True:
        read_data = fr.read(16)
        print(read_data)
        if read_data == b'':
            print('End of File')
            break
fr.close()

b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f'
b'\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'
b' !"#$%&\'()*+,-./'
b'0123456789:;<=>?'
b''
End of File


#### 05. 指定byteずつ読み込んだ任意の文字列を置換して出力する

In [69]:
# 初期化
header_size = 4
read_size = 16
input_file = './input.DMP'
output_file = './output/output.DMP'

target_from = b'\x00', b'\x10', b'\x20', b'\x30'
target_to = b'\xAB'

write_data = b''

# メイン処理
with open(input_file, 'rb') as fr:
    read_header = fr.read(header_size) #先頭4byteはスキップ  
    write_data = read_header
    while True:
        read_data = fr.read(read_size)
        print(read_data)
        for i in (target_from):
            read_data = read_data.replace(i, target_to)
        write_data += read_data
        if read_data == b'':
            print('End of File')
            break

#ファイル出力
fw = open(output_file, 'wb')
fw.write(write_data)
fw.close()
fr.close()

b'\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13'
b'\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#'
b"$%&'()*+,-./0123"
b'456789:;<=>?@ABC'
b'\x11'
b''
End of File


In [28]:
# データイメージ /1byte/
# aa/uu/xx/xx/ts/ts/ts/ts/1280byte data/

# 初期化
frame_header_size = 4 # DEADBEEF
line_header_size = 8 # ラインデータのヘッダ部
line_data_size = 4 # ラインデータ本体

input_file = './input.DMP'
output_file = './output/output.DMP'

target_unitID = b'\x03'
target_timestamp = b'\x04\x05\x06\x07'
target_from =  b'\xEF' # angleID=239
target_to = b'\xED' # angleID=237

count = 0
write_data = b''

# メイン処理
with open(input_file, 'rb') as fr:
    frame_header = fr.read(frame_header_size) #先頭4byteはスキップ  
    write_data = frame_header
    while True:
        line_header_data = fr.read(line_header_size)
        line_main_data = fr.read(line_data_size)
        if line_header_data[1:2] == target_unitID and line_header_data[4:8] == target_timestamp and \
        line_header_data[0:1] == target_from:
            count += 1
            line_header_replaced = line_header_data[0:2].replace(target_from, target_to)
        else:
            line_header_replaced = line_header_data[0:2]
        write_data += (line_header_replaced + line_header_data[2:8] + line_main_data)
        if line_header_data == b'' or line_main_data == b'' :
            print('End of File')
            break
                  
# ファイル出力
print("置換数:", count)
fw = open(output_file, 'wb')
fw.write(write_data)
fw.close()
fr.close()

End of File
置換数: 2


In [19]:
fr = open('input.DMP', 'rb')
read_data = fr.read()
print(read_data)
print(read_data[2:8])
fr.close()

b'\xef\xbe\xad\xde\x00\x01\xca\xcb\x04\x05\x06\x06\xaa\xaf\xaa\xaa\xef\x03\xef\xef\x04\x05\x06\x07\xef\xef\xef\xef\xef\x01\xef\xef\x04\x05\x06\x07\xef\xef\xef\xef\xaa\x03\xfe!\x04\x05\x06\x07\xad\xad\xad\xad'
b'\xad\xde\x00\x01\xca\xcb'


#### 06. GUI表示の作成イメージ

In [24]:
# tkinterのインポート
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import messagebox

# 関数birthdayの定義
# def birthday():
#     messagebox.showinfo("誕生日","あなたの誕生日は" + entry.get() + "です")
#     entry.delete(0,tk.END)

# rootフレームの設定
root = tk.Tk()
root.title("FileContentsConverter")
root.geometry("500x500")

# フレームの作成と設置
frame = ttk.Frame(root)
frame.grid(column=0, row=0, sticky=tk.NSEW, padx=5, pady=10)

# 各種ウィジェットの作成
label_header_size = ttk.Label(frame, text="ヘッダサイズ[byte]：", anchor=tk.E)
label_read_size = ttk.Label(frame, text="リードサイズ[byte]：", anchor=tk.E)
label_timestamp = ttk.Label(frame, text="timestamp：", anchor=tk.E)
label_unitnum = ttk.Label(frame, text="unit No.：", anchor=tk.E)
label_angle_before = ttk.Label(frame, text="置換前AngleID：", anchor=tk.E)
label_angle_after = ttk.Label(frame, text="置換後AngleID：", anchor=tk.E)

entry_header_size = ttk.Entry(frame)
entry_read_size = ttk.Entry(frame)
entry_timestamp = ttk.Entry(frame)
entry_unitnum = ttk.Entry(frame)
entry_angle_before = ttk.Entry(frame)
entry_angle_after = ttk.Entry(frame)

button_execute = ttk.Button(frame, text="実行", command=birthday)

# 各種ウィジェットの設置
label_header_size.grid(row=0, column=0)
label_read_size.grid(row=1, column=0)
label_timestamp.grid(row=2, column=0)
label_unitnum.grid(row=3, column=0)
label_angle_before.grid(row=4, column=0)
label_angle_after.grid(row=5, column=0)


entry_header_size.grid(row=0, column=2)
entry_read_size.grid(row=1, column=2)
entry_timestamp.grid(row=2, column=2)
entry_unitnum.grid(row=3, column=2)
entry_angle_before.grid(row=4, column=2)
entry_angle_after.grid(row=5, column=2)

button_execute.grid(row=10 , column=5)

# Entryウィジェットへ文字列のセット
# entry.insert(tk.END,"1999/01/01")

root.mainloop()

#### 07. 実行時に外部から与える変数を定義する

In [42]:
input_val = input('Enter header size(default 4):')
val = 4 if input_val == '' else int(input_val)

Enter header size(default 4): 6
